In [7]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import time

from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader, sampler
from torch import nn

from DatasetLoader import DatasetLoader
from Unet2D import Unet2D

In [19]:
def test(model, test_dl, acc_fn):
    model.cuda()
    model.eval()
    running_acc = 0
    for x, y in test_dl:
        x = x.cuda()
        y = y.cuda()
        with torch.no_grad():
            outputs = model(x)
            acc = acc_fn(outputs, y)

        running_acc += acc * test_dl.batch_size
    return running_acc

def acc_metric(predb, yb):
    return (predb.argmax(dim=1) == yb.cuda()).float().mean()

In [20]:
def main():
    params_path = "models/model_epoch_2.pth"
    bs = 12
    # load the test data
    # Current paths are incorrect
    base_path = Path('/work/datasets/medical_project/CAMUS_resized')
    data = DatasetLoader(base_path / 'train_gray',
                         base_path / 'train_gt')
    print(len(data))

    # split the training dataset and initialize the data loaders
    _, _, test_dataset = torch.utils.data.random_split(
        data,
        (300, 100, 50),
        generator=torch.Generator().manual_seed(42)
    )

    # split the training dataset and initialize the data loaders
    test_data = DataLoader(test_dataset, batch_size=bs, shuffle=False)

    xb, yb = next(iter(test_data))
    print(xb.shape, yb.shape)

    # build the Unet2D with one channel as input and 2 channels as output
    unet = Unet2D(1, 2)
    # Load best model params
    unet.load_state_dict(torch.load(params_path))
    # loss function and optimizer
    # loss_fn = nn.CrossEntropyLoss()
    # opt = torch.optim.Adam(unet.parameters())

    # do some training
    acc = test(unet, test_data, acc_metric)
    print(acc)

In [18]:
main()

450
torch.Size([12, 1, 384, 384]) torch.Size([12, 384, 384])
tensor(55.8430, device='cuda:0')
